## 11. Goal setting and monitoring

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import random
import re
from pathlib import Path
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

In [3]:
llm = ChatOpenAI(
   model="gpt-4o", # got-4o에 액세스할 수 없으면 다른 OpenAI LLM 사용
   temperature=0.3,
)

### 1. 유틸리티 함수

In [4]:
def generate_prompt(
   use_case: str, goals: list[str], previous_code: str = "", feedback: str = ""
) -> str:
   print("📝 코드 생성을 위한 프롬프트 구성 중...")
   base_prompt = f"""
You are an AI coding agent. Your job is to write Python code based on the following use case:
Use Case: {use_case}
Your goals are:
{chr(10).join(f"- {g.strip()}" for g in goals)}
"""
   if previous_code:
       print("🔄 개선을 위해 이전 코드를 프롬프트에 추가.")
       base_prompt += f"\nPreviously generated code:\n{previous_code}"
   if feedback:
       print("📋 수정을 위한 피드백 포함.")
       base_prompt += f"\nFeedback on previous version:\n{feedback}\n"
   base_prompt += "\nPlease return only the revised Python code. Do not include comments or explanations outside the code."
   return base_prompt
def get_code_feedback(code: str, goals: list[str]) -> str:
   print("🔍 목표에 대한 코드 평가 중...")
   feedback_prompt = f"""
You are a Python code reviewer. A code snippet is shown below. Based on the following goals:
{chr(10).join(f"- {g.strip()}" for g in goals)}
Please critique this code and identify if the goals are met. Mention if improvements are needed for clarity, simplicity, correctness, edge case handling, or test coverage.
Code:
{code}
"""
   return llm.invoke(feedback_prompt)

In [5]:
def goals_met(feedback_text: str, goals: list[str]) -> bool:
   """
   피드백 텍스트에 기반하여 목표가 충족되었는지 평가하기 위해 LLM을 사용합니다.
   True 또는 False를 반환합니다 (LLM 출력에서 파싱됨).
   """
   review_prompt = f"""
You are an AI reviewer.
Here are the goals:
{chr(10).join(f"- {g.strip()}" for g in goals)}
Here is the feedback on the code:
{feedback_text}
Based on the feedback above, have the goals been met?
Respond with only one word: True or False.
"""
   response = llm.invoke(review_prompt).content.strip().lower()
   return response == "true"

In [6]:
def clean_code_block(code: str) -> str:
   lines = code.strip().splitlines()
   if lines and lines[0].strip().startswith("```"):
       lines = lines[1:]
   if lines and lines[-1].strip() == "```":
       lines = lines[:-1]
   return "\n".join(lines).strip()

In [7]:
def add_comment_header(code: str, use_case: str) -> str:
   comment = f"# This Python program implements the following use case:\n# {use_case.strip()}\n"
   return comment + "\n" + code
def to_snake_case(text: str) -> str:
   text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
   return re.sub(r"\s+", "_", text.strip().lower())
def save_code_to_file(code: str, use_case: str) -> str:
   print("💾 최종 코드를 파일에 저장 중...")
   summary_prompt = (
       f"Summarize the following use case into a single lowercase word or phrase, "
       f"no more than 10 characters, suitable for a Python filename:\n\n{use_case}"
   )
   raw_summary = llm.invoke(summary_prompt).content.strip()
   short_name = re.sub(r"[^a-zA-Z0-9_]", "", raw_summary.replace(" ", "_").lower())[:10]
   random_suffix = str(random.randint(1000, 9999))
   filename = f"{short_name}_{random_suffix}.py"
   filepath = Path.cwd() / filename
   with open(filepath, "w") as f:
       f.write(code)
   print(f"✅ 코드가 다음에 저장됨: {filepath}")
   return str(filepath)

In [8]:
def run_code_agent(use_case: str, goals_input: str, max_iterations: int = 5) -> str:
   goals = [g.strip() for g in goals_input.split(",")]
   print(f"\n🎯 사용 사례: {use_case}")
   print("🎯 목표:")
   for g in goals:
       print(f"  - {g}")
   previous_code = ""
   feedback = ""
   for i in range(max_iterations):
       print(f"\n=== 🔁 {max_iterations} 중 {i + 1}회 반복 ===")
       prompt = generate_prompt(use_case, goals, previous_code, feedback if isinstance(feedback, str) else feedback.content)
       print("🚧 코드 생성 중...")
       code_response = llm.invoke(prompt)
       raw_code = code_response.content.strip()
       code = clean_code_block(raw_code)
       print("\n🧾 생성된 코드:\n" + "-" * 50 + f"\n{code}\n" + "-" * 50)
       print("\n📤 피드백 검토를 위해 코드 제출 중...")
       feedback = get_code_feedback(code, goals)
       feedback_text = feedback.content.strip()
       print("\n📥 받은 피드백:\n" + "-" * 50 + f"\n{feedback_text}\n" + "-" * 50)
       if goals_met(feedback_text, goals):
           print("✅ LLM이 목표가 충족되었음을 확인합니다. 반복 중지.")
           break
       print("🛠️ 목표가 완전히 충족되지 않음. 다음 반복 준비 중...")
       previous_code = code
   final_code = add_comment_header(code, use_case)
   return save_code_to_file(final_code, use_case)

### 2. 테스트

In [9]:
if __name__ == "__main__":
   print("\n🧠 AI 코드 생성 에이전트에 오신 것을 환영합니다")
   # 예제 1
   use_case_input = "Write code to find BinaryGap of a given positive integer"
   goals_input = "Code simple to understand, Functionally correct, Handles comprehensive edge cases, Takes positive integer input only, prints the results with few examples"
   run_code_agent(use_case_input, goals_input)
   # 예제 2
   # use_case_input = "Write code to count the number of files in current directory and all its nested sub directories, and print the total count"
   # goals_input = (
   #     "Code simple to understand, Functionally correct, Handles comprehensive edge cases, Ignore recommendations for performance, Ignore recommendations for test suite use like unittest or pytest"
   # )
   # run_code_agent(use_case_input, goals_input)
   # 예제 3
   # use_case_input = "Write code which takes a command line input of a word doc or docx file and opens it and counts the number of words, and characters in it and prints all"
   # goals_input = "Code simple to understand, Functionally correct, Handles edge cases"
   # run_code_agent(use_case_input, goals_input)


🧠 AI 코드 생성 에이전트에 오신 것을 환영합니다

🎯 사용 사례: Write code to find BinaryGap of a given positive integer
🎯 목표:
  - Code simple to understand
  - Functionally correct
  - Handles comprehensive edge cases
  - Takes positive integer input only
  - prints the results with few examples

=== 🔁 5 중 1회 반복 ===
📝 코드 생성을 위한 프롬프트 구성 중...
🚧 코드 생성 중...

🧾 생성된 코드:
--------------------------------------------------
def binary_gap(n):
    binary_representation = bin(n)[2:]
    max_gap = 0
    current_gap = 0
    found_one = False

    for digit in binary_representation:
        if digit == '1':
            if found_one:
                max_gap = max(max_gap, current_gap)
            current_gap = 0
            found_one = True
        else:
            current_gap += 1

    return max_gap

def main():
    examples = [9, 529, 20, 15, 32, 1041, 1]
    for number in examples:
        print(f"Binary gap of {number} is {binary_gap(number)}")

if __name__ == "__main__":
    main()
-----------------------------------